In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

# Load model and tokenizer
model_name = "arunyasenadeera/mT5-thai-summarization-finetuned-v4"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Use GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# Strong summarization function (no post-fix)
def strong_summarize(text, max_input_length=512, max_output_length=700):
    instruction = (
        "โปรดสรุปข้อความต่อไปนี้อย่างกระชับและชัดเจน "
        "รักษาข้อมูลสำคัญให้ครบถ้วน เช่น ชื่อบุคคล ตำแหน่ง หน่วยงาน สถานที่ วันเดือนปี "
        "ห้ามเปลี่ยนแปลงข้อเท็จจริง ห้ามเพิ่มเนื้อหาใหม่ และห้ามตัดข้อมูลสำคัญ "
        "ห้ามละข้อมูลที่เกี่ยวกับการประชุม การเตรียมการ และภารกิจสนับสนุนที่สำคัญ "
        "ใช้ภาษาทางการที่เหมาะสมสำหรับรายงานราชการ."
    )

    input_text = instruction + "\n\n" + text

    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        truncation=True,
        max_length=max_input_length,
        padding="longest"
    ).to(device)

    with torch.no_grad():
        summary_ids = model.generate(
            inputs["input_ids"],
            max_length=max_output_length,
            num_beams=5,
            no_repeat_ngram_size=2,
            early_stopping=True
        )

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Example usage:
your_text = """
สศท.สปท. กําหนดจัดงานสัปดาห์วิชาการ สปท. ประจําปีงบประมาณ พ.ศ. 2567 ในหัวข้อ “นวัตกรรมเพื่อความมั่นคง และความปลอดภัยสาธารณะ” ระหว่างวันที่ 4-5 มิ.ย. 67 ณ สปท. โดยมีวัตถุประสงค์ เพื่อเผยแพร่นวัตกรรมเพื่อความมั่นคง และเสริมสร้างภาพลักษณ์ที่ดีของกองทัพและหน่วยงานที่เกี่ยวข้อง รวมทั้งส่งเสริมความร่วมมือและความสัมพันธ์อันดีระหว่างหน่วยงานทั้งภายใน กห. และภายนอก กห. ซึ่งมีผู้เข้าชมงาน ประกอบด้วย กําลังพล สปท. ผู้เข้ารับการอบรมหลักสูตรในความรับผิดชอบของ สปท. นักเรียนทหาร คณาจารย์ นิสิตนักศึกษาของสถาบันการศึกษาที่มีความร่วมมือทางวิชาการระหว่างกันและบุคคลทั่วไป
เพื่อให้การดําเนินงานฯเป็นไปด้วยความเรียบร้อย สศท.สปท. จึงขอเชิญ คณะทํางานฯ เข้าร่วมประชุมเพื่อเตรียมการจัดงานฯ ในวันที่ 7 พ.ค. 67 เวลา 0900 ณ ห้อง 432 ชั้น 3 อาคารอเนกประสงค์ สปท. โดยมี พล.ท. สมยศ ฉันทวรลักษณ์ เสธ.สปท. เป็นประธานการประชุมฯ รายละเอียดตาม QR Code ด้านล่าง ทั้งนี้ ได้มอบหมายให้ ร.ท. ชัชรินทร์ เลิศยศบดินทร์ ประจําแผนกแผน และโครงการ กศษ.สศท.สปท. โทรศัพท์เคลื่อนที่ 08 2342 4267 เป็นผู้ประสานรายละเอียดการปฏิบัติ
"""

summary = strong_summarize(your_text)
print("📄 Summary:\n", summary)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/19.3k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/416 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/890 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

📄 Summary:
 สศท.สปท. ขอเชิญคณะทํางานฯ เข้าร่วมประชุมเพื่อเตรียมการจัดงานสัปดาห์วิชาการ สปท ประจําปีงบประมาณ พ.ศ. 2565 ในหัวข้อ “นวัตกรรมเพื่อความมั่นคง และความปลอดภัยสาธารณะ” ระหว่างวันที่ 4-5 มิ.ย. 67 เวลา  09 ณ ห้อง 432 ชั้น 3 อาคารอเนกประสงค์ สปชร. โดยมี พล.ท
